### 資料預處理

#### 資料預處理流程:

**遺失值處理(missing data)** $\rightarrow$ 資料轉換(transform) $\rightarrow$ 異常值處理(outlier) $\rightarrow$ 資料縮放(scaling)

### 遺失值(missing data)處理

#### 1. 為什麼存在遺失值?

- 真實世界的資料為雜亂的資料(messy data)

- 數值在資料獲取過程中遺失

- 數值意外地被刪除


#### 2. 為什麼要處理遺失值?

- 適當的處理遺失值可以降低引進誤差(bias)的機率

- 大部分 ML 演算法需要完整的資料


#### 3. 遺失值的處理流程

(1) 偵測所有的遺失值

(2) 轉換所有遺失值為 NaN 值

(3) 分析遺失值的數量和型態 (數值、時間序列、種類)

(4) 適當地刪除或填補遺失值

(5) 評估和比較資料處理過後的表現


#### 4. Null 值運算比較 (None v.s. np.nan)

(1) 邏輯運算:

- None: None or True/False $\Rightarrow$ True/False

- np.nan: np.nan or True/False $\Rightarrow$ nan

(2) 四則運算:

- None + True (For all operators) $\Rightarrow$ TypeError: unsupported operand

- np.nan * True (For all operators) $\Rightarrow$ nan

(3) 型態: type()

- type(None) $\Rightarrow$ NoneType

- type(np.nan) $\Rightarrow$ float

(4) 比較運算:

- None == None $\Rightarrow$ True

- np.nan == np.nan $\Rightarrow$ False

(5) np.isnan()

- np.isnan(None) $\Rightarrow$ False

- np.isnan(np.nan) $\Rightarrow$ True


#### 5. 偵測與取代遺失值

(1) 偵測遺失值: 

- character: "NA", "-", "."


- inherent missing values within the data like '0'


Example:

    df = pd.read_csv('data.csv')

    df.head()

    df.info()

    df_unique = df.csat.unique()
    np.sort(df_unique)


(2) 使用 NaN 取代遺失值:

a. '.' $\Rightarrow$ 'NaN'

Example:

    df = pd.read_csv('data.csv', na_values='.')

b. '0' $\Rightarrow$ 'NaN'

Example:

    df = pd.read_csv("data.csv')

    df.col[df.col == 0] = np.nan

    df.col[np.isnan(df.col)]
    
    
#### 6.  計算遺失值的數量

Example:

    df = pd.read_csv('data.csv', parse_dates=['Date'], index_col='Date')
    
    df.head()


(1) Nullity DataFrame: 使用 df.isnull() 或 df.isna() 的方法

    df_nullity = df.isnull()
    
    df_nullity.head()


(2) 遺失值總量

    df_nullity.sum()


(3) 遺失值佔所有資料的百分比

    df_nullity.mean() * 100


(4) Missingno package: 遺失值圖形化分析的套件

    import missingno as msno

- Nullity Bar: 

      msno.bar(df)


- Nullity Matrix: 

      msno.matrix(df)


- Nullity Matrix for time-series data: 

      msno.matrix(df, freq='M')


- Fine tuning the matrix: 

      msno.matrix(df.loc['start-date': 'end-date'], freq='M')


#### 7. 遺失值的型態

(1) 不同遺失值型態產生的原因

[Note] variable: data field or column in a DataFrame

- Values simply missing at random instances or intervals in a variable


- Values missing due to another variable


- Values missing due to the missingness of the same or another variable


(2) missingness 的型態

a. Missing Completely at Random (MCAR)

定義: Missingness has no relationship between any values, observed or missing.

b. Missing at Random (MAR): 

定義: There is a systematic relationship between missingness and other observed data, but not the missing data.

c. Missing Not at Random (MNAR): 

定義: There is a relationship between missingness and its values, missing or non-missing.


(3) 找出 missingness 之間的關係並分類

a. 排序變數值

    sorted = df.sort_values('col_name')
 
    msno.matrix(sorted)


b. Missingness heatmap or correlation map

    msno.heatmap(df)
   
   
- 圖形化欄位之間遺失值的相關性(correlation)


- 解釋欄位之間 missingness 的相依性


c. Missingness dendrogram

    msno.dendrogram(df)
    
    
- 以樹狀圖呈現 missingness


- 藉由分群變數來描述變數之間的關係


#### 8. 視覺化不同變數間的 missingness

Example:
    
    def fill_dummy_values(df, scaling_factor):
        # Create copy of dataframe
        df_dummy = df.copy(deep=True)

        # Iterate over each column
        for col in df_dummy:
      
            # Get column, column missing values and range
            col = df_dummy[col]
            col_null = col.isnull()
            num_nulls = col_null.sum()
            col_range = col.max() - col.min()

            # Shift and scale dummy values
            dummy_values = (rand(num_nulls) - 2)
            dummy_values = dummy_values * scaling_factor * col_range + col.min()

            # Return dummy values
            col[col_null] = dummy_values

        return df_dummy

    # Create dummy dataframe
    df_dummy = fill_dummy_values(df)

    # Get missing values of both columns for coloring
    nullity = df.col1.isnull() + df.col2.isnull()

    # Generate scatter plot
    df_dummy.plot(x='col1', y='col2', kind='scatter', alpha=0.5, c=nullity, cmap='rainbow')


#### 9. 刪除/省略(Omission)遺失值

(1) 刪除/省略(Omission)遺失值的條件

- 當遺失值的型態為 MCAR


(2) 刪除/省略(Omission)的方法

a. Pairwise deletion: 去除欄位有 NaN 的資料

Example 1: 去除所有含有遺失值的行

    df.dropna(axis=1)
    

Example 2: 計算平均值

    df['col'].mean() = df['col'].sum() / df['col'].count()


b. Listwise deletion: 去除整筆資料 

Example 1: 去除所有含有遺失值的列

    df.dropna(axis=0)


Example 2: 去除特定欄位含有遺失值的列

    df.dropna(subset=['col'], how='any', inplace=True)

    msno.matrix(df)


參數說明:

(i) how : {‘any’, ‘all’}, default ‘any’

- ‘any’ : If any NA values are present, drop that row or column.

- ‘all’ : If all values are NA, drop that row or column.


(ii) inplace : bool, default False

- If True, do operation inplace and return None.


#### 10. 基本的補值(imputation)方法與視覺化

(1) 平均數填補

    from sklearn.impute import SimpleImputer

    df_mean = df.copy(deep=True)
    
    mean_imputer = SimpleImputer(strategy='mean')
    
    df_mean.iloc[ : , : ] = mean_imputer.fit_transform(df_mean)


(2) 中位數填補

    df_median = df.copy(deep=True)

    median_imputer = SimpleImputer(strategy='median')
    
    df_median.iloc[ : , : ] = median_imputer.fit_transform(df_median)


(3) 眾數填補

    df_mode = df.copy(deep=True)

    mode_imputer = SimpleImputer(strategy='most_frequent')

    df_mode.iloc[ : , : ] = mode_imputer.fit_transform(df_mode)


(4) 常數填補

a. 使用 SimpleImputer() 函式

    df_constant = df.copy(deep=True)

    constant_imputer = SimpleImputer(strategy='constant', fill_value=0)

    df_constant.iloc[ : , : ] = constant_imputer.fit_transform(df_constant)


b. 使用 .fillna()

    df_filled = df.fillna(0)


(5) 迭代填補(Iterative imputation)

    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer

    # Subset numeric features: numeric_cols [字串欄位: df.select_dtypes(include = ['object'] )]
    numeric_cols = df.select_dtypes(include = [np.number])
 
    # Iteratively impute
    imp_iter = IterativeImputer(max_iter=5, sample_posterior=True, random_state=123)
    df_imp_iter = imp_iter.fit_transform(numeric_cols)

    # Convert returned array to DataFrame
    df_imp_iterDF = pd.DataFrame(df_imp_iter, columns=numeric_cols.columns)

    # Check the DataFrame's info
    print(df_imp_iterDF.info())


(6) 補值後的資料散佈圖(scatter plot)

    nullity = df['col_1'].isnull() + df['col_2'].isnull()

    df_mean.plot(x='col_1', y='col_2', kind='scatter', alpha=0.5, c=nullity, cmap='rainbow', title='Mean Imputation')


(7) 視覺化各種補值方法

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))

    nullity = df['col_1'].isnull() + df['col_2'].isnull()
    
    imputations = {'Mean Imputation': df_mean,
                   'Median Imputation': df_median,
                   'Most Frequent Imputation': df_mode,
                   'Constant Imputation': df_constant}

    for ax, df_key in zip(axes.flatten(), imputations):
    
        imputations[df_key].plot(x='col_1', y='col_2', kind='scatter', alpha=0.5, c=nullity, cmap='rainbow', ax=ax, colorbar=False, title=df_key)


(8) 各種刪除/省略(Omission)/填補方法對資料的影響:

- 刪除/省略(Omission): 可能移除大量的資料


- 以常數 0 填補: 誤差下降


- 平均數填補: 資料中異常值(outliers)的多寡會影響平均數的大小


- 中位數填補: 較適合處理異常值(outliers)存在的情況


- 眾數&迭代填補: 資料根據不同的情況會有不同程度的影響


#### 11. 時間序列資料的補值(imputation)與視覺化

(1) .fillna() method

a. Ffill/pad method

- Replace NaNs with last observed value

      df.fillna(method='ffill', inplace=True)

b. Bfill/backwardfill method

- Replace NaNs with next observed value

      df.fillna(method='bfill', inplace=True)

(2) .interpolate() method

a. Linear interpolation: Impute linearly or with equidistant values

    linear_interp = df.interpolate(method='linear', inplace=True)

b. Quadratic interpolation: Impute the values quadratically

    quadratic_interp = df.interpolate(method='quadratic', inplace=True)

c. Nearest value imputation: Impute with the nearest observable value

    nearest_interp = df.interpolate(method='nearest', inplace=True)

(3) Visualizing imputations 

Example: A comparison of the interpolations

    fig, axes = plt.subplots(3, 1, figsize=(30, 20))

    interpolations = {'Linear Interpolation': linear_interp,
                      'Quadratic Interpolation': quadratic_interp,
                      'Nearest Interpolation': nearest_interp}

    for ax, df_key in zip(axes, interpolations):
    
        interpolations[df_key].col.plot(color='red', marker='o', linestyle='dotted', ax=ax)
        
        df['col'].plot(title=df_key + ' - col', marker='o', ax=ax)
        
        
#### 12. 使用 fancyimpute 套件補值 (A variety of matrix completion and imputation algorithms)

(1) fancyimpute 套件

- 套件包含進階的填補技術


- 使用機械學習演算法(KNN)填補遺失值


- 使用其他的欄位特徵預測並填補遺失值(MICE)


(2) fancyimpute 填補技術

a. K-Nearest Neighbor (KNN)

- Select K nearest or similar data points using all the non-missing features


- Take average of the selected data points to fill in the missing feature

      
      from fancyimpute import KNN

      knn_imputer = KNN()

      df_knn = df.copy(deep=True)
      
      df_knn.iloc[ : , : ] = knn_imputer.fit_transform(df_knn)


b. Multiple Imputation by Chained Equations (MICE)

- Perform multiple regressions over random sample of the data


- Take average of the multiple regression values


- Impute the missing feature value for the data point


- MICE is a very robust model for imputation


      from fancyimpute import IterativeImputer

      MICE_imputer = IterativeImputer()

      df_MICE = df.copy(deep=True)

      df_MICE.iloc[ : , : ] = MICE_imputer.fit_transform(df_MICE)
      
      
#### 13. 填補種類值

(1) 種類值的複雜性(Complexity)

- 大多數的種類值為字串，不能對字串做運算


- 種類值必須從字串轉換到數值再填補


(2) 種類值的轉換方法

- one-hot encoder


- ordinal encoder


(3) 種類值的填補

- 以出現頻率最高的種類作填補


- 使用統計模型作填補(如: KNN)


(4) 填補種類值的步驟 (以 ordinal encoder 為例)

i. 將無遺失值的種類欄位轉換成 ordinal values


ii. 在 ordinal DataFrame 填補遺失值


iii. 將 ordinal values 轉換回種類值


**Step 1: Ordinal encoder**

    from sklearn.preprocessing import OrdinalEncoder
    
    # Create dictionary for Ordinal encoders
    ordinal_enc_dict = {}

    # Loop over columns to encode
    for col_name in df:
    
        # Create ordinal encoder for the column
        ordinal_enc_dict[col_name] = OrdinalEncoder()

        # Select the non-null values in the column
        col = df[col_name] 
        col_not_null = col[col.notnull()]
        reshaped_vals = col_not_null.values.reshape(-1, 1)

        # Encode the non-null values of the column
        encoded_vals = ordinal_enc_dict[col_name].fit_transform(reshaped_vals)

        # Replace the column with ordinal values
        df.loc[col.notnull(), 'col_name'] = np.squeeze(encoded_vals)
      
**Step 2: 使用 KNN 填補**

    df_KNN_imputed = df.copy(deep=True)

    # Create MICE imputer
    KNN_imputer = KNN()

    df_KNN_imputed.iloc[ : , : ] = np.round(KNN_imputer.fit_transform(df))

**Step 3: Ordinal decoder**

    for col_name in users:
        reshaped_col = df[col_name].values.reshape(-1, 1) 
        df_KNN_imputed[col_name] = ordinal_enc_dict[col_name].inverse_transform(reshaped_col)


#### 14. 評估不同的填補方法

(1) 為何要評估?

- 填補可以用來改進模型的表現


- 選擇能讓機械學習模型的表現為最佳的填補方法


- Density plots 能解釋了資料的分佈


- 可以作為檢查填補方法差異的度量


(2) 擬合線性模型並求出統計量

    import statsmodels.api as sm

    df_cc = df.dropna(how='any')

    X = sm.add_constant(df_cc.iloc[ : , :-1])

    y = df_cc['Class']

    lm = sm.OLS(y, X).fit()

    print(lm.summary())


R-squared 與 Coefficients:

- lm.rsquared_adj


- lm.params


(3) 評估與比較不同填補方法的步驟

i. 在 statsmodels 套件中使用線性模型


ii. 比較 coefficients 和 standard errors


iii. 比較 density plots

Example:

a. Fit linear model on different imputed DataFrames

    # Mean Imputation
    X = sm.add_constant(df_mean_imputed.iloc[:, :-1])
    y = df['Class']
    lm_mean = sm.OLS(y, X).fit()

    # KNN Imputation
    X = sm.add_constant(df_knn_imputed.iloc[:, :-1])
    lm_KNN = sm.OLS(y, X).fit()

    # MICE Imputation
    X = sm.add_constant(df_mice_imputed.iloc[:, :-1])
    lm_MICE = sm.OLS(y, X).fit()

b. Comparing R-squared of different imputations

    print(pd.DataFrame({'Complete': lm.rsquared_adj,
                        'Mean Imp.': lm_mean.rsquared_adj,
                        'KNN Imp.': lm_KNN.rsquared_adj,
                        'MICE Imp.': lm_MICE.rsquared_adj},
                         index=['R_squared_adj']))

c. Comparing coefficients of different imputations

    print(pd.DataFrame({'Complete': lm.params,
                        'Mean Imp.': lm_mean.params,
                        'KNN Imp.': lm_KNN.params,
                        'MICE Imp.': lm_MICE.params}))

d. Comparing density plots

    df_cc['A'].plot(kind='kde', c='red', linewidth=3)
    df_mean_imputed['A'].plot(kind='kde')
    df_knn_imputed['A'].plot(kind='kde')
    df_mice_imputed['A'].plot(kind='kde')

    labels = ['Baseline (Complete Case)', 'Mean Imputation', 'KNN Imputation', 'MICE Imputation']
 
    plt.legend(labels)
    plt.xlabel('A')